In [1]:
!pip install pyspark==3.4.2

In [2]:
!pip install minio

In [3]:
!pip install delta-spark==2.4.0

In [4]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F

In [5]:
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


24/08/27 20:59:50 WARN Utils: Your hostname, DESKTOP-CV9TLA8 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/27 20:59:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hacs/.cache/pypoetry/virtualenvs/pyspark-delta-gPM1PAZH-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hacs/.ivy2/cache
The jars for the packages stored in: /home/hacs/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-478afaa1-4656-4741-bb7e-b9340ce36cdd;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 129ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [6]:
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [7]:
objects=client.list_objects('raw', prefix='CAPES/')

In [8]:
objects=client.list_objects('raw', prefix='CAPES/')
csvs = []
for object in objects:
    csvs.append(object.object_name)

In [9]:
bucket_name = 'raw'

In [10]:
def equalslist(df1, df2):
    return list(set(df1.columns).intersection(set(df2.columns)))
    
def dfjoin(dataframe1, dataframe2, columnjoin, typejoin):
    
    equals = equalslist(dataframe1,dataframe2)
    equals.remove(columnjoin)
    dataframe3 = dataframe2
    for column in equals:
        dataframe3 = dataframe3.drop(column)
        
    dataframe_join = dataframe1.join(dataframe3, columnjoin, typejoin)
    dataframe_join = dataframe_join.dropna(subset=[columnjoin])
    return dataframe_join


In [11]:
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1

0 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
1 - CAPES/2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil.csv
2 - CAPES/2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil.csv
3 - CAPES/2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao.csv
4 - CAPES/2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu.csv
5 - CAPES/2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil.csv
6 - CAPES/2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil.csv
7 - CAPES/2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
8 - CAPES/2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
9 - CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil.csv
10 - CAPES/2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil.csv
11 - CAPE

In [12]:
autores = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[0]}')
teses = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[1]}')
cursos = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[2]}')
bibliografica = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[3]}')
tecnica = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[4]}')
discentes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[5]}')
docentes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[6]}')
financiadores = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[7]}')
membros = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[8]}')
producoes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[9]}')
programas = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[10]}')
projetos = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[11]}')

24/08/27 20:59:53 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [80]:
programas_cursos = dfjoin(programas, cursos, "CD_PROGRAMA_IES", "inner")
programas_projetos = dfjoin(programas, projetos, "CD_PROGRAMA_IES", "inner")
id_programas= dfjoin(programas_cursos, programas_projetos, "CD_PROGRAMA_IES", "inner")

In [14]:
projetos_producao = dfjoin(projetos, producoes, "ID_PROJETO", "inner")
projetos_financiadores = dfjoin(projetos, teses, "ID_PROJETO", "inner")
projetos_teses = dfjoin(projetos, teses, "ID_PROJETO", "inner")

In [15]:
financiadores_teses = dfjoin(projetos_financiadores, projetos_teses, "ID_PROJETO", 'inner')
id_projetos = dfjoin(projetos_producao, financiadores_teses, "ID_PROJETO", 'inner')

In [17]:
id_producao = dfjoin(producoes, autores, "ID_ADD_PRODUCAO_INTELECTUAL", "full")

In [48]:
df = dfjoin(id_producao, id_projetos, "ID_PROJETO", "inner")

In [49]:
df = dfjoin(df, id_programas, "CD_PROGRAMA_IES", "inner")

In [44]:
membros_discentes = dfjoin(discentes, membros, "ID_PESSOA", "left")

516580

In [46]:
membros_docentes = dfjoin(docentes, membros, "ID_PESSOA", "left")

In [51]:
df_discentes = dfjoin(df, membros_discentes, "ID_PROJETO", "inner")

97787844

In [53]:
df_docentes = dfjoin(df, membros_docentes, "ID_PROJETO", "inner")

In [64]:
discentes_ufma = df_discentes.filter(F.col("SG_ENTIDADE_ENSINO") == ("UFMA"))
discentes_ufma.count()

1512

In [74]:
discentes_ufma_tecnica = dfjoin(discentes_ufma, tecnica, "CD_PROGRAMA_IES", "left")

In [76]:
discentes_ufma_bibliografica = dfjoin(discentes_ufma, bibliografica, "CD_PROGRAMA_IES", "left")

In [78]:
docentes_ufma = df_docentes.filter(F.col("SG_ENTIDADE_ENSINO") == ("UFMA"))
docentes_ufma.count()

3274

In [79]:
docentes_ufma_tecnica = dfjoin(docentes_ufma, tecnica, "CD_PROGRAMA_IES", "left")
docentes_ufma_bibliografica = dfjoin(docentes_ufma, bibliografica, "CD_PROGRAMA_IES", "left")